In [82]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from stockfish import Stockfish
import chess

In [83]:
stockfish = Stockfish("./stockfish/stockfish_14_win_x64_avx2/stockfish_14_x64_avx2.exe", parameters={"Skill Level": 1})

In [84]:
move_indices = []

for from_square in range(64):
    for to_square in range(64):
        move_indices.append(chess.Move(from_square, to_square))

piece_dict = {
    'p' : [1,0,0,0,0,0,0,0,0,0,0,0],
    'P' : [0,0,0,0,0,0,1,0,0,0,0,0],
    'n' : [0,1,0,0,0,0,0,0,0,0,0,0],
    'N' : [0,0,0,0,0,0,0,1,0,0,0,0],
    'b' : [0,0,1,0,0,0,0,0,0,0,0,0],
    'B' : [0,0,0,0,0,0,0,0,1,0,0,0],
    'r' : [0,0,0,1,0,0,0,0,0,0,0,0],
    'R' : [0,0,0,0,0,0,0,0,0,1,0,0],
    'q' : [0,0,0,0,1,0,0,0,0,0,0,0],
    'Q' : [0,0,0,0,0,0,0,0,0,0,1,0],
    'k' : [0,0,0,0,0,1,0,0,0,0,0,0],
    'K' : [0,0,0,0,0,0,0,0,0,0,0,1],
    '.' : [0,0,0,0,0,0,0,0,0,0,0,0],
}

piece_value_dict = {
    'p' : -1,
    'P' : 1,
    'n' : -3,
    'N' : 3,
    'b' : -3,
    'B' : 3,
    'r' : -5,
    'R' : 5,
    'q' : -9,
    'Q' : 9
}

In [85]:
def board_to_matrix(board):
    lines = str(board).split("\n")
    matrix = []

    for line in lines:
        split_line = line.split(' ')
        matrix_line = list(map(lambda x: piece_dict[x], split_line))
        matrix.append(matrix_line)
    
    return matrix

def filter_legal_moves(move_probs, chess_board):
    filter_mask = np.zeros(move_probs.shape)
    
    legal_moves = chess_board.legal_moves
    for move in legal_moves:
        filter_mask[0][64*move.from_square + move.to_square] = 1

    return move_probs*filter_mask

def count_material(chess_board):
    board_string = str(chess_board)
    material_value = 0

    for k,v in piece_value_dict.items():
        material_value += board_string.count(k) * v

    return material_value



        

In [86]:

num_actions = 4096


# def create_q_model():
#     # Network defined by the Deepmind paper
#     inputs = layers.Input(shape=(8, 8, 12))

#     # Convolutions on the frames on the screen
#     layer1 = layers.Conv2D(32, 2, strides=(2, 2), activation="relu")(inputs)
#     layer2 = layers.Conv2D(64, 2, strides=(2, 2), activation="relu")(layer1)
#     layer3 = layers.Conv2D(128, 2, strides=(2, 2), activation="relu")(layer2)

#     layer4 = layers.Flatten()(layer3)

#     action = layers.Dense(num_actions, activation="linear")(layer4)

#     return keras.Model(inputs=inputs, outputs=action)     

def create_q_model():
    # Network defined by the Deepmind paper
    inputs = layers.Input(shape=(8, 8, 12))
    initializer = tf.keras.initializers.HeNormal()

    # Convolutions on the frames on the screen
    # layer1 = layers.Conv2D(32, 2, strides=(2, 2), activation="relu")(inputs)
    # layer2 = layers.Conv2D(64, 2, strides=(2, 2), activation="relu")(layer1)
    # layer3 = layers.Conv2D(64, 2, strides=(2, 2), activation="relu")(layer2)

    
    layer1 = layers.Dense(64, activation="relu", kernel_initializer=initializer)(inputs)
    layer2 = layers.Dense(128, activation="relu", kernel_initializer=initializer)(layer1)
    layer3 = layers.Flatten()(layer2)

    action = layers.Dense(num_actions, activation="linear")(layer3)

    return keras.Model(inputs=inputs, outputs=action)    

In [87]:

model = create_q_model()
model.load_weights('./model_weights/v4')

In [88]:
f = open('./data/test_data.epd', 'r')
test_data = []
for line in f.readlines():
    # duplicate lines to play as both white and black
    test_data.append(line)
    test_data.append(line)

In [89]:

num_games = 100

game_counter = 0
win_counter = 0
loss_counter = 0



play_as_white = False



In [90]:
for fen_position in test_data:
    game_counter += 1
    move_counter = 0


    # init stockfish
    stockfish = Stockfish("./stockfish/stockfish_14_win_x64_avx2/stockfish_14_x64_avx2.exe")

    # init board
    stockfish.set_fen_position(fen_position)
    chess_board = chess.Board(fen_position)

    play_as_white = not play_as_white

    stockfish_to_move = play_as_white

    done = False


    while True: # play

        # print(stockfish.get_board_visual())
        # print(chess_board)

        move_counter += 1

        stockfish_to_move = not stockfish_to_move


        if stockfish_to_move:
            try:

                best_move = stockfish.get_best_move()
                chess_board.push(chess.Move.from_uci(best_move))
                stockfish.make_moves_from_current_position([best_move])

            except:
                break
                

                    
            if chess_board.is_game_over():
                if chess_board.outcome().termination == chess.Termination.CHECKMATE:
                    loss_counter += 1
                break

            continue
        


        # transform board state to matrix
        board_matrix = np.array(board_to_matrix(chess_board))

        move = None
        move_num = 0

        state_tensor = tf.convert_to_tensor(board_matrix)
        state_tensor = tf.expand_dims(state_tensor, 0)
        move_probs = model(state_tensor, training=False)
        # Take best action
        move_probs = filter_legal_moves(move_probs, chess_board)
        move_num = tf.argmax(move_probs[0]).numpy()

        # transform move_num to move, add promotion if possible
        promotion = None
        from_square = move_num // 64
        to_square = move_num % 64
        if from_square > 47 and from_square < 56 and chess_board.piece_at(from_square) == 'P':
            promotion = 5
        if from_square > 7 and from_square < 16 and chess_board.piece_at(from_square) == 'p':
            promotion = 5
        move = chess.Move(from_square, to_square, promotion)

        # Apply the sampled move in our environment
        chess_board.push(move)
        stockfish.make_moves_from_current_position([str(move)])


        if chess_board.is_checkmate():
            win_counter += 1


        if chess_board.is_game_over():
            break


    print(f"Game {game_counter} ended in {move_counter} moves, wins: {win_counter}, loses: {loss_counter}, draws: {game_counter - win_counter - loss_counter}.")


Game 1 ended in 36 moves, wins: 0, loses: 1, draws: 0.
Game 2 ended in 43 moves, wins: 0, loses: 2, draws: 0.
Game 3 ended in 12 moves, wins: 0, loses: 3, draws: 0.
Game 4 ended in 61 moves, wins: 0, loses: 4, draws: 0.
Game 5 ended in 16 moves, wins: 0, loses: 5, draws: 0.
Game 6 ended in 25 moves, wins: 0, loses: 6, draws: 0.
Game 7 ended in 116 moves, wins: 0, loses: 7, draws: 0.
Game 8 ended in 23 moves, wins: 0, loses: 8, draws: 0.
Game 9 ended in 42 moves, wins: 0, loses: 9, draws: 0.
Game 10 ended in 39 moves, wins: 0, loses: 10, draws: 0.
Game 11 ended in 38 moves, wins: 0, loses: 11, draws: 0.
Game 12 ended in 55 moves, wins: 0, loses: 11, draws: 1.
Game 13 ended in 30 moves, wins: 0, loses: 12, draws: 1.
Game 14 ended in 33 moves, wins: 0, loses: 13, draws: 1.
Game 15 ended in 28 moves, wins: 0, loses: 14, draws: 1.
Game 16 ended in 85 moves, wins: 0, loses: 15, draws: 1.
Game 17 ended in 42 moves, wins: 0, loses: 16, draws: 1.
Game 18 ended in 21 moves, wins: 0, loses: 17, d